# Hugging Face 数据集目录说明

本仓库通过 `datasets.load_dataset("Bingsu/Cat_and_Dog")` 拉取数据，并在 `dataset/` 下生成两个关键目录。理解它们的职责有助于后续管理：

- `hf_cache/`：🤗 Datasets 的自动缓存。包含下载的压缩包、解包后的 `dataset.arrow` 分片、`state.json` 等元数据。库内部使用哈希结构命名，**不要手工改动**，否则缓存命中与增量更新都会失效。
- `Cat_and_Dog/`：调用 `DatasetDict.save_to_disk()` 后的序列化副本，保留当前所有 split 及其 schema。目录中每个 split 拥有 `data-00000-of-00001.arrow` 与 `dataset_info.json` 等文件，可通过 `load_from_disk` 跨平台共享。

如果希望导出原始图片，可遍历 `DatasetDict` 并手动调用 `Image.save()` 写入其他文件夹；对缓存和磁盘副本本身不做修改。

## 为什么是 .arrow 格式？

- **列式存储**：Apache Arrow 以列为单位组织数据，支持内存映射（memory mapping），无需一次性载入即可高效随机访问，非常适合大规模训练数据。
- **零拷贝管道**：常用的 `map`、`filter`、`shuffle` 操作直接作用于 Arrow 缓冲区，减少 Python 层循环与序列化成本，并能更快地在 CPU/GPU 之间传输。
- **跨语言生态**：Arrow 标准被 Pandas、PySpark、DuckDB 等工具复用，`.arrow` 文件可以被其他数据工程组件直接读取，利于与下游流程对接。
- **统一特征定义**：无论原始是图像、文本还是结构化表，Arrow 会连同 `dataset_info.json` 一起保存 schema 和元数据；图像特征会引用缓存中的实际文件路径或字节流，兼顾可读性与性能。

因此，我们同时保留缓存与磁盘副本：缓存保证重复加载时无需重新下载，磁盘副本则便于共享、备份和版本化管理。

## 读取 `data-00000-of-00001.arrow` 的示例

`dataset/Cat_and_Dog/train` 中的 Arrow 文件存储了训练 split 的所有样本。下面演示几种常见的访问方式。

In [ ]:
from datasets import load_from_disk
from pathlib import Path

root = Path('~/PycharmProjects/DeepLearning').expanduser()
train_ds = load_from_disk(root / 'dataset/Cat_and_Dog')['train']
train_ds

### 1. 使用 `dataset[0]['image']` 直接拿到 `PIL.Image`

🤗 Datasets 会自动解码 `Image` 特征。我们可以直接获取 PIL 对象或转 Tensor：

In [ ]:
from torchvision import transforms

sample = train_ds[0]  # 访问第 0 个样本, 可以直接获取图片和标签
pil_image = sample['image']  # 直接获取 PIL 图片
label = sample['labels']  # 直接获取标签
tensor_image = transforms.ToTensor()(pil_image)  # 转换为 Tensor
label, pil_image, tensor_image.shape

### 2. 保存样本到磁盘

需要将 Arrow 中的图片导出成 PNG/JPEG，可用 PIL 保存：

In [ ]:
output_dir = root / 'dataset' / 'export_train_images'
output_dir.mkdir(parents=True, exist_ok=True)

for idx in range(5):  # 示例导出前 5 张
    sample = train_ds[idx]
    img = sample['image']
    label = sample['labels']
    img.save(output_dir / f'{idx}_{label}.png')

### 3. 转为 PyTorch / TensorFlow / NumPy dataloader

🤗 Datasets 内置 `with_format` 与 `to_tf_dataset` 等方法，可直接转成框架兼容的迭代器。下面展示 PyTorch 写法：

In [ ]:
import torch

train_pt = train_ds.with_format('torch')
batch = train_pt[:4]
batch['image'].shape, batch['labels']

### 4. 利用 `map` 批量处理

可以使用 `map` 在 Arrow 层做批量处理，并配合 `batched=True` 加速。示例：为图片添加随机翻转后再写回磁盘。

In [ ]:
from torchvision.transforms import RandomHorizontalFlip

flip = RandomHorizontalFlip(p=1.0)

def augment(batch):
    images = [flip(img) for img in batch['image']]
    batch['image'] = images
    return batch

aug_ds = train_ds.map(augment, batched=True, batch_size=32)
aug_ds

> **注意**：所有方法最终都是通过 🤗 Datasets 读取 Arrow 文件；无需自行解析二进制内容，从而保持安全与高效。

## 常见数据加载方案对比

| 来源 | 底层格式 | 典型入口 | 特点 | 适用场景 |
| --- | --- | --- | --- | --- |
| 🤗 Datasets | Apache Arrow（`*.arrow` + 元数据） | `datasets.load_dataset` / `load_from_disk` | 支持懒加载、矢量化 `map`、自动解码多模态特征；跨语言生态好 | 需要快速迭代、跨框架共享的大规模数据 |
| torchvision.datasets | 原始文件（PNG/JPEG、二进制批次等） | `torchvision.datasets.CIFAR10` 等 | 直接返回 `(PIL | Tensor, label)`，与 `DataLoader` 无缝衔接；通常内置下载脚本 | 传统 CV 任务，依赖 PyTorch 生态 |
| torchtext / torchaudio | 文本：原始语料；音频：wav/flac | `torchtext.datasets.AG_NEWS` 等 | 针对特定模态定制处理流水线 | NLP / 音频任务 |
| 自定义文件夹 | JPG/PNG + 自定义标签 | `ImageFolder` / `Dataset` 子类 | 最大自由度，需要自行维护缓存/索引 | 企业内部私有数据 |

**关键差异**：🤗 Datasets 倚赖 Arrow 提供统一 schema 与高效管道；`torchvision.datasets` 等传统接口更贴近 PyTorch `Dataset`/`DataLoader` 语义，直接围绕文件夹或压缩包构建。两者可以互转：Arrow 数据经过 `with_format('torch')` 可直接塞入 `DataLoader`，`torchvision` 数据也能转成 `Dataset.from_dict` 再推到 Hugging Face Hub。

### torchvision.datasets 示例

以下代码演示如何使用 `torchvision.datasets.CIFAR10` 读取数据。运行时如本地无缓存会自动下载：

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

transform = transforms.ToTensor()
cifar_train = CIFAR10(root=root / 'torchvision_data', train=True, download=False, transform=transform)
len(cifar_train), cifar_train[0][0].shape, cifar_train.classes[:5]

### 互操作建议

- 如果已有 Arrow 数据，需要在 PyTorch 中训练，可通过 `with_format('torch')` 或 `set_format` 直接返回 Tensor。
- 若要把 torchvision 数据集上传到 Hugging Face Hub，可遍历 `Dataset`，转成 `datasets.Dataset.from_dict` 或 `Dataset.from_generator`。
- 大型项目通常将原始文件存一份（便于复现），再派生出 Arrow / TFRecord / LMDB 等多种格式，视训练管道选择对应加载器。